In [ ]:
import os,sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append('..')
sys.path.append('.')
if not os.path.exists('VGDFR'):
    os.chdir('../')
# print current work dir
print(os.getcwd())
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from hyvideo.vae.autoencoder_kl_causal_3d import AutoencoderKLCausal3D
from hyvideo.vae import load_vae
from hyvideo.utils.file_utils import save_videos_grid
from hyvideo.inference import HunyuanVideoSampler
from pathlib import Path
from datetime import datetime
import time
from hyvideo.config import *
from VGDFR.hunyuan_vgdfr import VGDFRHunyuanVideoSampler,mod_rope_forward


In [ ]:
string_args="""--video-size 544 960 --video-length 65 --infer-steps 50 --prompt cat. --flow-reverse --use-cpu-offload --save-path ./results --seed 3"""
string_args=string_args.split(" ")
print(string_args)

def parse_args_new(namespace=None,string_args=None):
    parser = argparse.ArgumentParser(description="HunyuanVideo inference script")

    parser = add_network_args(parser)
    parser = add_extra_models_args(parser)
    parser = add_denoise_schedule_args(parser)
    parser = add_inference_args(parser)
    parser = add_parallel_args(parser)

    args = parser.parse_args(string_args,namespace=namespace)
    args = sanity_check_args(args)

    return args

args = parse_args_new(string_args=string_args)
print(args)


In [ ]:
models_root_path = Path(args.model_base)
hunyuan_video_sampler = VGDFRHunyuanVideoSampler.from_pretrained(models_root_path, args=args)

In [ ]:
seed = 3
args.infer_steps = 50
width, height = 960, 544
width, height = 500, 344
video_length = 97
prompt="A spirited individual rides a vintage bicycle along a sunlit, tree-lined path, wearing a casual outfit of a white t-shirt, denim shorts, and sneakers. The scene captures the golden hour, with sunlight filtering through the leaves, casting dappled shadows on the ground. The rider's hair flows freely in the breeze, and a joyful smile lights up their face. As they pedal, the camera zooms in to reveal the intricate details of the bike's design, including its classic handlebars and shiny bell. The background features a serene park with blooming flowers and a distant lake, enhancing the sense of freedom and tranquility."
for before_compression_steps in [5, 10, 15]:
    hunyuan_video_sampler.pipeline.before_compression_steps = before_compression_steps
    for similarity_threshold in [0.6, 0.7, 0.8, 0.9]:
        hunyuan_video_sampler.pipeline.similarity_threshold = similarity_threshold

        samples = hunyuan_video_sampler.predict(
            prompt=prompt,
            height=height,
            width=width,
            video_length=video_length,
            seed=seed,
            negative_prompt=args.neg_prompt,
            infer_steps=args.infer_steps,
            guidance_scale=args.cfg_scale,
            num_videos_per_prompt=args.num_videos,
            flow_shift=args.flow_shift,
            batch_size=args.batch_size,
            embedded_guidance_scale=args.embedded_cfg_scale,
        )["samples"]
        save_path = args.save_path
        # log_dlfr_t = hunyuan_video_sampler.pipeline.log_dlfr_t
        # Save samples
        if "LOCAL_RANK" not in os.environ or int(os.environ["LOCAL_RANK"]) == 0:
            for i, sample in enumerate(samples):
                sample = samples[i].unsqueeze(0)
                time_flag = datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d-%H:%M:%S")
                file_name = f"raw_seed{seed}_{prompt[:100].replace('/','')}"
                raw_save_path = (
                    f"{save_path}/example/th_{similarity_threshold}_k{before_compression_steps}/{file_name}.mp4"
                )
                save_videos_grid(sample, raw_save_path, fps=12)
                torch.save(
                    sample,
                    f"{save_path}/example/th_{similarity_threshold}_k{before_compression_steps}/{file_name}.pt",
                )


 60%|██████    | 27/45 [00:23<00:15,  1.17it/s]